In [2]:
import boto3
import pandas as pd
import numpy as np
import os
import logging
import boto3
from botocore.exceptions import ClientError
import os
from dotenv import load_dotenv
from const import aws_variables

In [3]:
!pwd

/home/iffi/Videos/private_key_commands/S3_operation


# Uploading file to S3 

In [4]:
def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param fiAWS_S3_BUCKETle_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = os.path.basename(file_name)

    # Upload the file
    s3_client = boto3.client('s3', 
                             region_name=aws_variables.S3_REGION,
                            aws_access_key_id=aws_variables.AWS_ACCESS_KEY_ID,
                            aws_secret_access_key=aws_variables.AWS_SECRET_ACCESS_KEY)
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

bucket =  aws_variables.S3_BUCKET
file_name = 'EMA2233M10102_model_response.csv'
upload_file(file_name,bucket,'sample.csv')


True

## Downloading Files from S3

In [5]:
s3_client = boto3.client('s3', 
                             region_name=aws_variables.S3_REGION,
                            aws_access_key_id=aws_variables.AWS_ACCESS_KEY_ID,
                            aws_secret_access_key=aws_variables.AWS_SECRET_ACCESS_KEY
                        )

s3_client.download_file(aws_variables.S3_BUCKET,
                       'sample.csv', # filename or path on s3
                        'sample.csv' # filename or path you want to save on local file
                       )

## Reading files from s3

In [6]:
s3_client = boto3.client('s3', 
                             region_name=aws_variables.S3_REGION,
                            aws_access_key_id=aws_variables.AWS_ACCESS_KEY_ID,
                            aws_secret_access_key=aws_variables.AWS_SECRET_ACCESS_KEY)

response = s3_client.get_object(Bucket=aws_variables.S3_BUCKET,
                               Key = 'sample.csv'
                               )
if response['ResponseMetadata']['HTTPStatusCode'] ==200:
        print(f"Successful S3 get_object response")
        data =  pd.read_csv(response.get('Body'))
        print(data.head())

Successful S3 get_object response
   shot_id            shot_time     COUNTER_ID  DATA_ID_DC  cycle_time  \
0      1.0  2024-03-19 21:55:28  EMA2233M10102     6321190          30   
1      2.0  2024-03-19 21:55:59  EMA2233M10102     6321190          31   
2      3.0  2024-03-19 21:56:30  EMA2233M10102     6321190          30   
3      4.0  2024-03-19 21:57:01  EMA2233M10102     6321190          30   
4      5.0  2024-03-19 21:57:32  EMA2233M10102     6321190          31   

     temp  value1  value2  similarityMetric  similarityMetricHr  \
0  35.800       1       0                 0                   0   
1  35.794       1       0                 0                   0   
2  35.789       1       0                 0                   0   
3  35.783       1       0                 0                   0   
4  35.777       1       0                 0                   0   

   cycletime_upper_threshold  cycletime_lower_threshold  ACCELERATIONS  
0                         33                 

## reading files from s3 using s3fs api

In [7]:
temp_df = pd.read_csv(
    f"s3://{aws_variables.S3_BUCKET}/sample.csv",
    storage_options={
            "key": aws_variables.AWS_ACCESS_KEY_ID,
            "secret": aws_variables.AWS_SECRET_ACCESS_KEY,
        },
)
temp_df.head()

,shot_id,shot_time,COUNTER_ID,DATA_ID_DC,cycle_time,temp,value1,value2,similarityMetric,similarityMetricHr,cycletime_upper_threshold,cycletime_lower_threshold,ACCELERATIONS
0,1.0,2024-03-19 21:55:28,EMA2233M10102,6321190,30,35.800,1,0,0,0,33,27,NaN
1,2.0,2024-03-19 21:55:59,EMA2233M10102,6321190,31,35.794,1,0,0,0,33,27,NaN
2,3.0,2024-03-19 21:56:30,EMA2233M10102,6321190,30,35.789,1,0,0,0,33,27,NaN
3,4.0,2024-03-19 21:57:01,EMA2233M10102,6321190,30,35.783,1,0,0,0,33,27,NaN
4,5.0,2024-03-19 21:57:32,EMA2233M10102,6321190,31,35.777,1,0,0,0,33,27,NaN


## Write pandas dataframe to csv file on s3

In [8]:
import io

books_df = pd.DataFrame(
    data={"Title": ["Book I", "Book II", "Book III"], "Price": [56.6, 59.87, 74.54]},
    columns=["Title", "Price"],
)


with io.StringIO() as csv_buffer:
    books_df.to_csv(csv_buffer, index=False)

    response = s3_client.put_object(
        Bucket=aws_variables.S3_BUCKET, Key="files/books.csv", Body=csv_buffer.getvalue()
    )

    status = response.get("ResponseMetadata", {}).get("HTTPStatusCode")

    if status == 200:
        print(f"Successful S3 put_object response. Status - {status}")
    else:
        print(f"Unsuccessful S3 put_object response. Status - {status}")


Successful S3 put_object response. Status - 200


## See thes filesnames or path on s3 buckets

In [46]:
import boto3
from botocore.exceptions import NoCredentialsError, PartialCredentialsError

def list_s3_objects(bucket_name, folder_prefix):
    """
    List all objects in a specified folder within an S3 bucket.

    :param bucket_name: The name of the S3 bucket
    :param folder_prefix: The folder prefix to list objects from
    :return: A list of object keys
    """
    try:
        s3= boto3.client('s3', 
                             region_name=aws_variables.S3_REGION,
                            aws_access_key_id=aws_variables.AWS_ACCESS_KEY_ID,
                            aws_secret_access_key=aws_variables.AWS_SECRET_ACCESS_KEY
                        )

        # Paginator to handle large result sets
        paginator = s3.get_paginator('list_objects_v2')

        # Create a PageIterator from the Paginator with the specified prefix
        page_iterator = paginator.paginate(Bucket=bucket_name, Prefix=folder_prefix)

        # List to store object keys
        object_keys = []

        # Iterate through each page of results
        for page in page_iterator:
            if 'Contents' in page:
                for obj in page['Contents']:
                    object_keys.append(obj['Key'])

        return object_keys

    except NoCredentialsError:
        print("Error: No AWS credentials found.")
        return None
    except PartialCredentialsError:
        print("Error: Incomplete AWS credentials.")
        return None
    except Exception as e:
        print(f"Error: {e}")
        return None

# Example usage
if __name__ == "__main__":
    bucket_name = aws_variables.S3_BUCKET
    folder_prefix = 'Datasets/extracted_data'  # e.g., 'folder/subfolder/'
    objects = list_s3_objects(bucket_name, folder_prefix)
    if objects is not None:
        print(f"Objects in folder '{folder_prefix}' in bucket '{bucket_name}':")
        for obj in objects:
            print(obj)
            break


Objects in folder 'Datasets/extracted_data' in bucket 'tft-bucket-v1':
Datasets/extracted_data/
